# Phase 2i: IQtree Building

Generates distance tree using IQTree.

<details>
    <summary>Click To See A Decription of Parameters</summary>
        <pre>
            <code>
save_dir: str
    Path to directory for saving outputs in.

fasta_path: str
    Name used for fasta files containing sequences.

max_threads: int, default None
    The maximum number of threads to use when calling gnu parallel.

  </code>
</pre>

In [ ]:
save_dir=''
fasta_path= ''
max_threads=''

In [ ]:
# For some reason this still has to be called so that slurm's sbatch to use the beast_pype environment.
source activate beast_pype

In [ ]:
default_threads=11
if [ $max_threads -gt $default_threads ]; then
    cores_to_use=$default_threads
    jobs_per_time=$((max_threads / default_threads))
else
    cores_to_use=$max_threads
    jobs_per_time=1
fi

In [ ]:
if [ -f "${fasta_path}" ]; then
    mkdir ${save_dir}/initial_trees
    iqtree -ninit 2 -n 2 -me 0.05 -nt $cores_to_use -s ${fasta_path} -m GTR -ninit 10 -n 4\
            --prefix ${save_dir}/initial_trees/iqtree > ${save_dir}/initial_trees/iqtree.out
    mv ${save_dir}/initial_trees/iqtree.treefile ${save_dir}/initial_trees/iqtree.nwk
else
    valid_dirs=$(find $save_dir -type d)
    array_to_remove=($save_dir $save_dir/.ipynb_checkpoints)
    for element in "${array_to_remove[@]}"
        do
        valid_dirs=("${valid_dirs[@]/$element}")
        done
    mkdir -v "{valid_dirs[@]/%/initial_trees}"
    parallel --jobs=$jobs_per_time --results {1}/initial_trees/iqtree_\
        iqtree -ninit 2 -n 2 -me 0.05 -nt $cores_to_use -s {1}/${fasta_path} -m GTR -ninit 10 -n 4\
        --prefix {1}/initial_trees/iqtree\
        ::: ${valid_dirs[@]}
    for directory in ${valid_dirs[@]}
        do
        mv ${directory}/initial_trees/iqtree.treefile ${directory}/initial_trees/iqtree.nwk
        done
fi